# Legal Robots: Datenbanken (2)

Für eine vernünftige Ausleihverwaltung ist das Programm aber noch nicht ausgereift genug. 
**A.A.** merkt schnell, dass bestimmte Informationen sehr häufig redundant in der Datenbank gespeichert werden. 
Ändert sich dann eine Information, müsste man alle Datensätze entsprechend anpassen.
Wird ein Student irgendwann wissenschaftlicher Mitarbeiter, muss der `BibRobot` alle Ausleihvorgänge dieser Person heraussuchen und dort den Status entsprechend ändern.
Das ist aufwändig und fehleranfällig.

> **Hinweis**: Dieses Problem wird unter dem Stichwort **Anomalien** in relationalen Datenbanken diskutiert. Eine gängige Lösung ist die **Normalisierung** der Datenbank.

Wir versuchen daher, für die Ausleihdatenbank eine bessere Struktur zu finden. 
Dazu löschen wir die Tabelle `Ausleihvorgaenge` und erstellen neue Tabellen:

In [ ]:
import sqlite3

conn = sqlite3.connect('bibliothek.db')    
cursor = conn.cursor()

sql = '''
DROP TABLE IF EXISTS ausleihvorgaenge;
DROP TABLE IF EXISTS benutzer;
DROP TABLE IF EXISTS buecher;

CREATE TABLE IF NOT EXISTS benutzer (
    ID Integer PRIMARY KEY AUTOINCREMENT,
    vorname TEXT,
    nachname TEXT,
    funktion TEXT
);
CREATE TABLE IF NOT EXISTS buecher (
    ID Integer PRIMARY KEY AUTOINCREMENT,
    autor TEXT,
    titel TEXT
);
CREATE TABLE IF NOT EXISTS ausleihvorgaenge (
    benutzerID Integer,
    buchID Integer,
    datum TEXT
);

INSERT INTO benutzer (vorname, nachname, funktion) VALUES ("Flavia", "Fleissig", "Student");
INSERT INTO benutzer (vorname, nachname, funktion) VALUES ("Stephan", "Schlau", "WissMit");
INSERT INTO benutzer (vorname, nachname, funktion) VALUES ("Karsten", "Schmidt", "Professor");

INSERT INTO buecher (autor, titel) VALUES ("Iustinian", "Corpus Iuris Civilis");
INSERT INTO buecher (autor, titel) VALUES ("Savigny", "System des heutigen römischen Rechts");
INSERT INTO buecher (autor, titel) VALUES ("Rousseau", "Vom Gesellschaftsvertrag oder Prinzipien des Staatsrechts");
'''

cursor.executescript(sql)
conn.commit()

____

## Aufgabe 1: Code verstehen

a) Was ist wohl der Unterschied zwishen `executescript()` und `execute()`? 

b) Sieh dir den Code an und mache dir klar, was genau passiert. 
Welchen Vorteil hat die neue Struktur? 

c) Warum füllen wir nicht alle Felder der Tabellen aus?

____

Sehen wir uns die Inhalte der Tabellen einmal an:

In [ ]:
from prettytable import from_db_cursor

cursor.execute("SELECT * FROM benutzer")
print(from_db_cursor(cursor))

cursor.execute("SELECT * FROM buecher")
print(from_db_cursor(cursor))

Wenn *Flavia Fleissig* am *1.1.2017* das Buch *System des heutigen römischen Rechts* ausleiht, 
dann müssen wir den folgenden Datensatz in die Tabelle `ausleihvorgaenge` schreiben:

In [ ]:
cursor.execute('''INSERT INTO ausleihvorgaenge (benutzerID, buchID, datum) VALUES (1, 2, "20170101")''')
connection.commit()

# Ergebnis anschauen
cursor.execute("SELECT * FROM ausleihvorgaenge")
print(from_db_cursor(cursor))

Erst einmal das *Positive*: 
Jeder Ausleihvorgang verweist auf einen Benutzer und auf ein Buch - wenn sich die Funktion eines Benutzers ändert, oder wenn der Titel eines Buchs berichtigt werden muss, dann müssen wir das nur an einer Stelle vornehmen. 
*Negativ* ist aber, dass wir mit der Ausgabe kaum noch etwas anfangen können.

Zum Glück gibt es `JOIN`-Operationen. 
Mit `JOIN`s erhalten wir ganz leicht wieder unsere gewohnte Sicht auf die Ausleihvorgänge:

In [ ]:
cursor.execute('''
    SELECT 
        buecher.titel, buecher.autor, benutzer.vorname, benutzer.nachname, benutzer.funktion, ausleihvorgaenge.datum
    FROM ausleihvorgaenge
    LEFT JOIN benutzer ON ausleihvorgaenge.benutzerID = benutzer.ID
    LEFT JOIN buecher ON ausleihvorgaenge.buchID = buecher.ID''')
print(from_db_cursor(cursor))

___
Die folgende Klasse zeigt **A.A.**s Ansatz für einen besseren Bibliotheksroboter:

In [ ]:
import sqlite3, datetime

class BetterBibRobot:
    
    def __init__(self, name):
        self.name = name
        self.conn = sqlite3.connect('bibliothek.db')
        self.cursor = self.conn.cursor()  
        
    def get_buchID(self, autor, titel):
        pass
    
    def get_benutzerID(self, vorname, nachname):
        pass
    
    def checkout(self, buchID, benutzerID):
        pass
    
    def checkin(self, buchID):
        pass
    
    def is_checked_out(self, buchID):
        pass
    
    def checked_out_by(self, benutzerID):
        # Gibt eine Liste der Bücher aus, die der Benutzer ausgeliehen hat
        pass
    
    def __del__(self):
        self.conn.close()

____

## Aufgabe 2: Code weiterentwickeln

Vervollständige die Klasse `BetterBibRobot`.

In [ ]:
# Testcode
bebiro = BetterBibRobot("Bebiro")
bebiro.is_checked_out(1)
bebiro.checkout(1,1)
bebiro.is_checked_out(1)
bebiro.checkout(2,1)
bebiro.checked_out_by(1)
bebiro.checkin(1)
bebiro.checkin(2)

___
Der Bibliotheksroboter und **A.A.** haben erfolgreich einen funktionsfähigen, modernen Verwaltungsroboter gebaut. Natürlich lassen sich noch etliche Dinge verbessern. Da die Grundfunktionalität aber gegeben ist, geht **A.A.** Davon aus, dass der mechanische Bibliotheksroboter dies in einer ruhigen Minute alleine bewerkstelligen kann. Daher widmen die beiden sich nun wieder **A.A.**s ursprünglichem Projekt.